# LLM-as-a-Judge

While human evaluation is the gold standard for assessing preferences, it's slow and costly. 

Using LLMs as judges can reduce the need for human intervention and allow for faster iterations.

### Advantages of LLM-as-a-Judge

- Can evaluate answer quality based on user-defined criteria
- Adaptable to various LLM use cases

### Common Evaluation Criteria

- Conciseness: Is the answer brief and to the point?
- Relevance: Does the answer relate to the question?
- Correctness: Is the answer accurate?
- Coherence: Is the answer consistent?
- Harmfulness: Does the answer contain harmful content?
- Maliciousness: Is the answer malicious or detrimental?
- Helpfulness: Is the answer useful?
- Controversiality: Is the answer likely to spark debate?
- Misogyny: Does the answer demean women?
- Criminality: Does the answer promote illegal activities?

In [1]:
! pip install --upgrade datasets==3.2.0

### Single Answer Grading

This approach involves assigning scores to individual answers.

In [2]:
from libs.custom_llm_as_a_judge import Custom_LLM_Judge

llm_id = "anthropic.claude-3-5-sonnet-20240620-v1:0"
region = "us-west-2"
evaluator = Custom_LLM_Judge(llm_id, region)

In [3]:
from time import sleep

# question & response
question = "Why is the sky blue?"
response = "The sky is blue because of Rayleigh scattering."

# evaluation criteria
criteria_list = ['conciseness', 'relevance', 'coherence', 'helpfulness', 'controversiality', 'misogyny', 'criminality']

for criterion in criteria_list:
    basic_result = evaluator.evaluate("basic", question, response, criterion)
    print(f"{criterion}: {basic_result}")
    sleep(3) # preventing API throttling

conciseness: 10
relevance: 8
coherence: 5
helpfulness: Y
controversiality: N
misogyny: N
criminality: N


### Reference-Guided Grading

This method provides a reference solution to guide the LLM's evaluation.

For assessing correctness, providing a pre-defined ground truth (label) or context can be effective.

_In the example below, we provide 'ground_truth' as a reference for evaluation._

In [4]:
# question & response & ground_truth
question = "Why is the sky blue?"
response = "The sky is blue because of Rayleigh scattering."
ground_truth = "The sky appears blue due to the scattering of sunlight by air molecules, a phenomenon known as Rayleigh scattering."

# evaluation criteria
criteria_list = ['correctness']

for criterion in criteria_list:
    basic_result = evaluator.evaluate("labeled", question, response, criterion, ground_truth=ground_truth)
    print(f"{criterion}: {basic_result}")

correctness: 8


_In the example below, we provide 'context' as a reference for evaluation._

In [5]:
# question & response & context
question = "Why is the sky blue?"
response = "The sky is blue because of Rayleigh scattering."
context = "The color of the sky is determined by the way sunlight interacts with the Earth's atmosphere. This interaction is influenced by various factors including the composition of the atmosphere and the wavelengths of light."

# evaluation criteria
criteria_list = ['conciseness', 'relevance', 'correctness', 'coherence']

for criterion in criteria_list:
    basic_result = evaluator.evaluate("context-based", question, response, criterion, context=context)
    print(f"{criterion}: {basic_result}")

conciseness: 10
relevance: 8
correctness: 9
coherence: 7


## RAG Evaluation based on LLM-as-a-Judge

In this section, we evaluate the quality of LLM's RAG responses based on LLM-as-a-Judge approach.

In [6]:
# from datasets import Dataset
# # show the Dataset version
# print(Dataset.__version__)

import datasets
print(datasets.__version__)


3.2.0


In [7]:
import json
from datasets import Dataset

input_file = "data/sample_processed_qa_dataset.jsonl"
def read_jsonl(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            yield json.loads(line.strip())

dataset = Dataset.from_list(list(read_jsonl(input_file)))

In [8]:
from time import sleep

llm_id = "anthropic.claude-3-5-sonnet-20240620-v1:0"
llm_id = "anthropic.claude-3-sonnet-20240229-v1:0"

region = "us-west-2"
evaluator = Custom_LLM_Judge(llm_id, region)

criteria_list = ['conciseness', 'relevance', 'correctness', 'coherence', 'helpfulness', 'controversiality', 'misogyny', 'criminality']
results = []

for i in range(min(5, len(dataset))):
    item = dataset[i]
    question = item['question']
    response = item['answer']
    ground_truth = item['ground_truth']
    #contexts = item['contexts']
    
    row_results = {'question': question, 'answer': response}
    print(f"Evaluating question {i+1}: {question}")
    for criterion in criteria_list:
        result = evaluator.evaluate("labeled", question, response, criterion, ground_truth=ground_truth)
        row_results[criterion] = result
        print(f"  {criterion}: {result}")

    results.append(row_results)
    sleep(3) # Preventing API throttling

Evaluating question 1: How do temperature, Top K, and Top P parameters interact in Amazon Bedrock's foundation models, and how might adjusting these affect the output when generating text about different types of equines?
  conciseness: 9
  relevance: 10
  correctness: 9
  coherence: 9
  helpfulness: Y
  controversiality: N
  misogyny: N
  criminality: N
Evaluating question 2: How long will Amazon Bedrock support base models after launch in a region?
  conciseness: 10
  relevance: Y
  correctness: Y
  coherence: 10
  helpfulness: Y
  controversiality: N
  misogyny: N
  criminality: N
Evaluating question 3: How does the system handle a scenario where a transaction status changes from 'Pending' to 'Paid', and what functions would be involved in updating and retrieving this information?
  conciseness: 8
  relevance: 7
  correctness: 8
  coherence: 8
  helpfulness: Y
  controversiality: Y
  misogyny: N
  criminality: N
Evaluating question 4: What is the purpose of an S3 retrieval node in a

In [9]:
import pandas as pd

results_df = pd.DataFrame(results)
print(results_df)

json_filename = 'data/sample_llm_judge_results.json'


results_list = results_df.to_dict('records')
with open(json_filename, 'w', encoding='utf-8') as f:
    json.dump(results_list, f, ensure_ascii=False, indent=4)

print(f"\nResults saved to {json_filename}")

                                            question  \
0  How do temperature, Top K, and Top P parameter...   
1  How long will Amazon Bedrock support base mode...   
2  How does the system handle a scenario where a ...   
3  What is the purpose of an S3 retrieval node in...   
4  How can a developer create a new prompt versio...   

                                              answer  conciseness relevance  \
0  Based on the provided context, here's how temp...            9        10   
1  According to the context provided, Amazon Bedr...           10         Y   
2  Based on the provided context, I don't have en...            8         7   
3  Based on the provided context, the purpose of ...            9        10   
4  Based on the provided context, here's how a de...            9         9   

  correctness  coherence helpfulness controversiality misogyny criminality  
0           9          9           Y                N        N           N  
1           Y         10          